# Monte Carlo Modeling with PyMC3

This notebook demonstrates how to build and use a Monte Carlo model using the `pymc` library. The model is applied to weather data to predict a target variable, and the results are visualized.

## Define the MonteCarloModel Class

Define a class `MonteCarloModel` that initializes with features (X) and target variable (y). It includes methods for fitting the model using `pymc` and making predictions.

In [ ]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

class MonteCarloModel():
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.model = None
        self.trace = None

    def fit(self):
        with pm.Model() as model:
            shift = pm.Normal("shift", mu=0, sigma=1, shape=self.X.shape[0])
            scale = pm.HalfNormal("scale", sigma=1, shape=self.X.shape[0])
            bias = pm.Normal("bias", mu=0, sigma=1)  # Single bias term
            sigma = pm.HalfNormal("sigma", sigma=1)  # Single sigma term
            mu = pm.Deterministic("mu", (self.X.T - shift) * scale + bias)
            y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=self.y)
            trace = pm.sample(1000, tune=1000)
        self.model = model
        self.trace = trace

    def predict(self, X):
        with self.model:
            ppc = pm.sample_posterior_predictive(self.trace, var_names=["mu"], samples=1000)
            predictions = ppc.posterior["mu"].mean(axis=(0,1))  # Average over samples and features
        return predictions

## Fit the Monte Carlo Model

Instantiate the `MonteCarloModel` with the prepared features and target variable. Fit the model using the `fit` method, which defines the model structure, priors, and performs MCMC sampling.

In [ ]:
# Sample Data (Replace with your actual data loading and preprocessing)
# Example data:
num_features = 6
num_samples = 200
features = np.random.rand(num_features, num_samples)
y = np.random.rand(num_samples)

# Split data into training and testing
train_size = int(0.7 * num_samples)
train_features = features[:, :train_size]
test_features = features[:, train_size:]
train_y = y[:train_size]
test_y = y[train_size:]

mc_model = MonteCarloModel(train_features, train_y)
print("Fitting")
mc_model.fit()

## Make Predictions with the Monte Carlo Model

Use the trained `MonteCarloModel` to make predictions on both the training and test sets. Visualize the predictions against the actual values using `matplotlib`.

In [ ]:
# Make predictions on training and test sets
train_pred = mc_model.predict(train_features)
test_pred = mc_model.predict(test_features)

# Plot training predictions
plt.figure(figsize=(12, 6))
plt.title("Training Predictions")
plt.plot(train_pred, label="Predicted")
plt.plot(train_y, label="Actual")
plt.legend()
plt.show()

# Plot test predictions
plt.figure(figsize=(12, 6))
plt.title("Test Predictions")
plt.plot(test_pred, label="Predicted")
plt.plot(test_y, label="Actual")
plt.legend()
plt.show()